# Comparison of Pre-trained Models for Cat and Dog Classification

## 1. Setup and Imports

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import (
    EfficientNetB0, ResNet50, VGG16,
    efficientnet, resnet, vgg16
)
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import requests
from PIL import Image
from io import BytesIO

In [ ]:
# Constants
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10
TRAIN_DIR = './dataset_dogs_vs_cats/train'
TEST_DIR = './dataset_dogs_vs_cats/test'
MODEL_SAVE_DIR = './saved_models'
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

## 2. Data Loading and Preprocessing

In [ ]:
# Create data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

## 3. Model Creation and Training

### 3.1. EfficientNetB0

In [ ]:
# Create EfficientNetB0 model
base_effnet = EfficientNetB0(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

# Freeze the base model
base_effnet.trainable = False

# Create new model on top
inputs = tf.keras.Input(shape=(224, 224, 3))
x = efficientnet.preprocess_input(inputs)
x = base_effnet(x, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation='sigmoid')(x)

effnet_model = Model(inputs, outputs)

# Compile the model
effnet_model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the model
effnet_history = effnet_model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=[
        EarlyStopping(patience=3, restore_best_weights=True),
        ModelCheckpoint(
            os.path.join(MODEL_SAVE_DIR, 'efficientnet_best.h5'),
            save_best_only=True,
            monitor='val_accuracy',
            mode='max'
        )
    ]
)

### 3.2. ResNet50

In [ ]:
# Create ResNet50 model
base_resnet = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

# Freeze the base model
base_resnet.trainable = False

# Create new model on top
inputs = tf.keras.Input(shape=(224, 224, 3))
x = resnet.preprocess_input(inputs)
x = base_resnet(x, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation='sigmoid')(x)

resnet_model = Model(inputs, outputs)

# Compile the model
resnet_model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the model
resnet_history = resnet_model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=[
        EarlyStopping(patience=3, restore_best_weights=True),
        ModelCheckpoint(
            os.path.join(MODEL_SAVE_DIR, 'resnet50_best.h5'),
            save_best_only=True,
            monitor='val_accuracy',
            mode='max'
        )
    ]
)

### 3.3. VGG16

In [ ]:
# Create VGG16 model
base_vgg = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

# Freeze the base model
base_vgg.trainable = False

# Create new model on top
inputs = tf.keras.Input(shape=(224, 224, 3))
x = vgg16.preprocess_input(inputs)
x = base_vgg(x, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation='sigmoid')(x)

vgg_model = Model(inputs, outputs)

# Compile the model
vgg_model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the model
vgg_history = vgg_model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=[
        EarlyStopping(patience=3, restore_best_weights=True),
        ModelCheckpoint(
            os.path.join(MODEL_SAVE_DIR, 'vgg16_best.h5'),
            save_best_only=True,
            monitor='val_accuracy',
            mode='max'
        )
    ]
)

## 4. Model Evaluation

In [ ]:
# Evaluate models on test set
effnet_test_loss, effnet_test_acc = effnet_model.evaluate(test_generator)
resnet_test_loss, resnet_test_acc = resnet_model.evaluate(test_generator)
vgg_test_loss, vgg_test_acc = vgg_model.evaluate(test_generator)

print(f"EfficientNetB0 Test Accuracy: {effnet_test_acc:.4f}")
print(f"ResNet50 Test Accuracy: {resnet_test_acc:.4f}")
print(f"VGG16 Test Accuracy: {vgg_test_acc:.4f}")

In [ ]:
# Plot training history
def plot_history(history, model_name):
    plt.figure(figsize=(12, 4))
    
    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'{model_name} - Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'{model_name} - Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Plot history for each model
plot_history(effnet_history, 'EfficientNetB0')
plot_history(resnet_history, 'ResNet50')
plot_history(vgg_history, 'VGG16')

## 5. Testing with Downloaded Images

In [ ]:
def test_with_downloaded_images(model, class_indices):
    # Example image URLs
    test_images = {
        'cat1': 'https://cdn.pixabay.com/photo/2017/02/20/18/03/cat-2083492_1280.jpg',
        'cat2': 'https://cdn.pixabay.com/photo/2014/11/30/14/11/cat-551554_1280.jpg',
        'dog1': 'https://cdn.pixabay.com/photo/2016/12/13/05/15/puppy-1903313_1280.jpg',
        'dog2': 'https://cdn.pixabay.com/photo/2018/01/09/11/04/dog-3071334_1280.jpg',
    }
    
    plt.figure(figsize=(15, 10))
    
    for i, (name, url) in enumerate(test_images.items(), 1):
        try:
            # Download and preprocess image
            response = requests.get(url)
            img = Image.open(BytesIO(response.content)).resize(IMG_SIZE)
            img_array = np.array(img) / 255.0
            img_array = np.expand_dims(img_array, axis=0)
            
            # Make prediction
            prediction = model.predict(img_array)[0][0]
            predicted_class = 'dog' if prediction > 0.5 else 'cat'
            confidence = prediction if predicted_class == 'dog' else (1 - prediction)
            
            # Plot image with prediction
            plt.subplot(2, 2, i)
            plt.imshow(img)
            plt.title(f'Predicted: {predicted_class}\nConfidence: {confidence:.2f}')
            plt.axis('off')
            
        except Exception as e:
            print(f"Error processing {url}: {e}")
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Test with the best model
# Determine the best model
accuracies = {
    'EfficientNetB0': effnet_test_acc,
    'ResNet50': resnet_test_acc,
    'VGG16': vgg_test_acc
}

best_model_name = max(accuracies, key=accuracies.get)
best_model = {
    'EfficientNetB0': effnet_model,
    'ResNet50': resnet_model,
    'VGG16': vgg_model
}[best_model_name]

print(f"\nBest model: {best_model_name} with test accuracy: {accuracies[best_model_name]:.4f}")

# Test with downloaded images
print("\nTesting with downloaded images...")
test_with_downloaded_images(best_model, test_generator.class_indices)

## 6. Save the Best Model

In [ ]:
# Save the best model
best_model.save(os.path.join(MODEL_SAVE_DIR, 'best_model.h5'))
print(f"Best model saved to {os.path.join(MODEL_SAVE_DIR, 'best_model.h5')}")
print("You can now load this model using: model = tf.keras.models.load_model('saved_models/best_model.h5')")

## 7. Model Comparison and Conclusions

In [ ]:
# Compare model performances
plt.figure(figsize=(10, 6))
plt.bar(accuracies.keys(), accuracies.values(), color=['blue', 'green', 'red'])
plt.title('Model Comparison - Test Accuracy')
plt.ylabel('Accuracy')
plt.ylim(0, 1.1)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Key Observations and Conclusions
1. **Performance Comparison**:
   - **EfficientNetB0**: {:.2f}% test accuracy
   - **ResNet50**: {:.2f}% test accuracy
   - **VGG16**: {:.2f}% test accuracy

2. **Best Performing Model**:
   - **{}** achieved the highest accuracy of {:.2f}% on the test set.

3. **Training Observations**:
   - [Add your observations about training stability, convergence speed, etc.]
   - [Note any overfitting/underfitting issues]
   - [Compare training times and computational requirements]

4. **Recommendations**:
   - [Suggest potential improvements or next steps]
   - [Discuss limitations and future work]

5. **Final Thoughts**:
   [Add your overall conclusions and thoughts on the project]